In [0]:
import pandas as pd
import numpy as np
import os
import datetime
import matplotlib.pyplot as plt
from skimage import color, exposure
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

In [4]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_signs'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_signs


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')


X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train,y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train,
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_train,y_train)),
      callbacks=[tensorboard_callback]
  )
  return model

def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)       #true values

  y_pred_prob = model_trained.predict(X_test)   #predicted values
  y_pred = np.argmax(y_pred_prob, axis=1)       #shows where is the biggest probability

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu',padding='same'),
    MaxPool2D(),
    Dropout(0.3),
    
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),


    Conv2D(filters=64, kernel_size=(3, 3), activation='relu',padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),


    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),


    Dense(num_classes, activation='softmax'),
])

In [9]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 5s 17ms/step - loss: 2.7662 - accuracy: 0.2623 - val_loss: 1.2209 - val_accuracy: 0.6117
Epoch 2/5
272/272 [==============================] - 4s 16ms/step - loss: 0.9858 - accuracy: 0.6865 - val_loss: 0.3230 - val_accuracy: 0.9098
Epoch 3/5
272/272 [==============================] - 4s 16ms/step - loss: 0.4819 - accuracy: 0.8481 - val_loss: 0.1409 - val_accuracy: 0.9606
Epoch 4/5
272/272 [==============================] - 4s 15ms/step - loss: 0.3346 - accuracy: 0.8972 - val_loss: 0.1135 - val_accuracy: 0.9675
Epoch 5/5
272/272 [==============================] - 4s 15ms/step - loss: 0.2454 - accuracy: 0.9254 - val_loss: 0.0661 - val_accuracy: 0.9821


0.9541950113378684

In [10]:
model_trained.evaluate(X_test, y_test)[1]

138/138 [==============================] - 0s 4ms/step - loss: 0.1630 - accuracy: 0.9542


0.9541950225830078

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu',padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),
    
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),


    Conv2D(filters=64, kernel_size=(3, 3), activation='relu',padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),


    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),


    Dense(num_classes, activation='softmax'),
])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0
  )
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [36]:
space = {
    'batch_size': hp.quniform('batch_size', 50, 200, 20),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.2, 0.6),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.2, 0.6),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.2, 0.6),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.2, 0.6),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.2, 0.6)


}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 100.0, 'dropout_cnn_block_one': 0.25058605712400944, 'dropout_cnn_block_three': 0.45943531942519583, 'dropout_cnn_block_two': 0.30100859418169046, 'dropout_dense_block_one': 0.2024703012286162, 'dropout_dense_block_two': 0.2117312287813828}
accuracy=0.9655328989028931
{'batch_size': 60.0, 'dropout_cnn_block_one': 0.4804852370565796, 'dropout_cnn_block_three': 0.34183324319368424, 'dropout_cnn_block_two': 0.26093626218821847, 'dropout_dense_block_one': 0.43477389492104734, 'dropout_dense_block_two': 0.47004495304380606}
accuracy=0.9410430788993835
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.5487734127001462, 'dropout_cnn_block_three': 0.29719307836866893, 'dropout_cnn_block_two': 0.2702989565715414, 'dropout_dense_block_one': 0.2916686326012327, 'dropout_dense_block_two': 0.4722795709618532}
accuracy=0.8954648375511169
{'batch_size': 60.0, 'dropout_cnn_block_one': 0.5126378397460919, 'dropout_cnn_block_three': 0.3728687638432876, 'dropout_cnn_block_two': 0.4133375650